# NTDS'18 milestone 1: network collection and properties
[Effrosyni Simou](https://lts4.epfl.ch/simou), [EPFL LTS4](https://lts4.epfl.ch)

## Students

* Team: 31
* Students: Dilara Günay, Derin Sinan Bursa, Othman Benchekroun, Sinan Gökçe
* Dataset: IMDb Films and Crew

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to three sentences.
* Code has to be clean.
* You cannot import any other library than we imported.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective 

The purpose of this milestone is to start getting acquainted to the network that you will use for this class. In the first part of the milestone you will import your data using [Pandas](http://pandas.pydata.org) and you will create the adjacency matrix using [Numpy](http://www.numpy.org). This part is project specific. In the second part you will have to compute some basic properties of your network. **For the computation of the properties you are only allowed to use the packages that have been imported in the cell below.** You are not allowed to use any graph-specific toolboxes for this milestone (such as networkx and PyGSP). Furthermore, the aim is not to blindly compute the network properties, but to also start to think about what kind of network you will be working with this semester. 

In [ ]:
import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Part 1 - Import your data and manipulate them. 

###  A. Load your data in a Panda dataframe.

First, you should define and understand what are your nodes, what features you have and what are your labels. Please provide below a Panda dataframe where each row corresponds to a node with its features and labels. For example, in the the case of the Free Music Archive (FMA) Project, each row of the dataframe would be of the following form:


| Track   |  Feature 1  | Feature 2 | . . . | Feature 518|  Label 1 |  Label 2 |. . .|Label 16|
|:-------:|:-----------:|:---------:|:-----:|:----------:|:--------:|:--------:|:---:|:------:|
|         |             |           |       |            |          |          |     |        |

It is possible that in some of the projects either the features or the labels are not available. This is OK, in that case just make sure that you create a dataframe where each of the rows corresponds to a node and its associated features or labels.

In [ ]:
movies = pd.read_csv('data/tmdb_5000_credits.csv')
movies = movies[movies.cast != '[]']
movies.drop([813, 1891, 2862, 2905], axis=0, inplace=True)
movies.head()

In [ ]:
def flatten():
    movies['cast_id'] = movies['cast'].apply(lambda row: list(pd.read_json(row)['id']))
    movies['cast_id'] = movies['cast_id'].apply(lambda x: list(set(x)))
    movies.drop(['title', 'crew'], axis=1, inplace=True)

In [ ]:
flatten()
movies.head()

In [ ]:
discount = movies['cast_id'].apply(pd.Series).stack().value_counts()
discount = discount[discount > 4].index.astype(int)
discount[:10]

In [ ]:
movies['cast_id'] = movies['cast_id'].apply(lambda x: [y for y in x if y in discount])
movies['edges'] = movies['cast_id'].apply(lambda x: list(itertools.combinations(x, 2)))
edges = list(movies['edges'].apply(pd.Series).stack())
edges[0:10]

In [ ]:
frames = pd.DataFrame()
new_df = pd.DataFrame()

for idx, film in movies.iterrows():
    cast_df = pd.DataFrame(eval(movies['cast'][idx]))
    cast_df['movies'] = idx
    cast_df = cast_df.drop(['character','order','cast_id', 'credit_id'],axis = 1)  
    
    frames = [new_df, cast_df]
    new_df = pd.concat(frames, join = 'outer', ignore_index=True)

In [ ]:
nodes_df = new_df['movies'].groupby([new_df.gender, new_df.id, new_df.name]).apply(list).reset_index()
nodes_df = nodes_df[nodes_df.id.isin(discount)]
nodes_df.head()

In [ ]:
features = nodes_df.set_index('id').drop('name', axis=1)
features.head()

### B. Create the adjacency matrix of your network.

Remember that there are edges connecting the attributed nodes that you organized in the dataframe above. The connectivity of the network is captured by the adjacency matrix $W$. If $N$ is the number of nodes, the adjacency matrix is an $N \times N$ matrix where the value of $W(i,j)$ is the weight of the edge connecting node $i$ to node $j$.  

There are two possible scenarios for your adjacency matrix construction, as you already learned in the tutorial by Benjamin:

1) The edges are given to you explicitly. In this case you should simply load the file containing the edge information and parse it in order to create your adjacency matrix. See how to do that in the  [graph from edge list]() demo.

2) The edges are not given to you. In that case you will have to create a feature graph. In order to do that you will have to chose a distance that will quantify how similar two nodes are based on the values in their corresponding feature vectors. In the [graph from features]() demo Benjamin showed you how to build feature graphs when using Euclidean distances between feature vectors. Be curious and explore other distances as well! For instance, in the case of high-dimensional feature vectors, you might want to consider using the cosine distance. Once you compute the distances between your nodes you will have a fully connected network. Do not forget to sparsify by keeping the most important edges in your network.

Follow the appropriate steps for the construction of the adjacency matrix of your network and provide it in the Numpy array ``adjacency`` below: 

In [ ]:
#DISCOUNT: DISCARD LESS THAN 5 MOVIES, CREW
adj = pd.DataFrame(np.zeros(shape=(len(discount),len(discount))), columns=discount, index=discount)
for e1, e2 in edges:
    adj.at[e1, e2] += 1
    adj.at[e2, e1] += 1
adj.head()

In [ ]:
adjacency = adj.values
adj_max = adjacency.max()
adjacency = np.vectorize(lambda x: x/adj_max)(adjacency)

In [ ]:
len(adjacency)

In [ ]:
n_nodes = len(discount)
n_nodes

## Part 2

Execute the cell below to plot the (weighted) adjacency matrix of your network.

In [ ]:
plt.spy(adjacency, markersize=0.005)
plt.title('adjacency matrix')

### Question 1

What is the maximum number of links $L_{max}$ in a network with $N$ nodes (where $N$ is the number of nodes in your network)? How many links $L$ are there in your collected network? Comment on the sparsity of your network.

In [ ]:
L_max = n_nodes*(n_nodes-1)/2
print(L_max)

In [ ]:
L_max / len(edges)

In [ ]:
n_nodes * math.sqrt(n_nodes)

**We have that the network is not fully connected, yet not sparse. Indeed, we have that $L=O(n*\sqrt{n})$.**

### Question 2

Is your graph directed or undirected? If it is directed, convert it to an undirected graph by symmetrizing the adjacency matrix.

**To make a matrix symmetric, we sum the original matrix with its transpose and divide the result by 2.**

In [ ]:
adjacency = (adjacency + adjacency.T)/2

### Question 3

In the cell below save the features dataframe and the **symmetrized** adjacency matrix. You can use the Pandas ``to_csv`` to save the ``features`` and Numpy's ``save`` to save the ``adjacency``. We will reuse those in the following milestones.

In [ ]:
np.save("data/adj.npy", adjacency)

### Question 4

Are the edges of your graph weighted?

**Yes, the edges are weighted and the weights correspond to the number of movies where actors played together.**

### Question 5

What is the degree distibution of your network? 

In [ ]:
degree = np.count_nonzero(adjacency, axis=1)

assert len(degree) == n_nodes

Execute the cell below to see the histogram of the degree distribution.

In [ ]:
weights = np.ones_like(degree) / float(n_nodes)
plt.hist(degree, weights=weights);

What is the average degree?

In [ ]:
np.sum(degree)/n_nodes

### Question 6

Comment on the degree distribution of your network.

**The degree distribution of the graph seems large but is not unexpected. Indeed, the number of actors in a movie is $8$ on average, while the number of movies an actor played in is $10$ in average. The order of magnitude seems to be corresponding according to the rule of thumb.**

### Question 7

Write a function that takes as input the adjacency matrix of a graph and determines whether the graph is connected or not.

In [ ]:
def connected_graph(adjacency):
    #in visited and queue, we store the indices of columns/rows
    visited, queue = set(), [0]
    while queue:
        index = queue.pop(0)
        if index not in visited:
            visited.add(index)
            actor_connected_to = set(np.nonzero(adjacency[index])[0])
            queue.extend(actor_connected_to - visited)
            
    if len(visited) == n_nodes:
        connected = True
    else:
        connected = False
    
    return connected

Is your graph connected? Run the ``connected_graph`` function to determine your answer.

In [ ]:
# Your code here.
if(connected_graph(adjacency)):
    print('Our graph is connected')
else:
    print('Our graph is not connected')

### Question 8

Write a function that extracts the connected components of a graph.

In [ ]:
def find_components(adjacency):
    not_visited = []
    for i in range(len(adjacency)):
        not_visited.append(i)
    #Before starting the while loop, not_visited is containing all the nodes (row indices in the adjacency matrix)
    
    component_list = list() #This list is created in order to contain the components
    while not_visited:
        queue = [not_visited[0]]
        visited = set()
        while queue:
            index = queue.pop(0)
            if index not in visited:
                visited.add(index)
                actor_connected_to = set(np.nonzero(adjacency[index])[0])
                queue.extend(actor_connected_to - set(visited))
                
        not_visited = list(set(not_visited) - visited)
        component_list.append(list(visited))
    
    components = []
    for i in range(len(component_list)):        
        component = []
        for row in component_list[i]:
            row_list = []
            for column in component_list[i]:
                row_list.append(adjacency[row][column])
            row_list = np.asarray(row_list)    
            component.append(row_list)
        component = np.asarray(component)      
        components.append(component)    
    
    return components

How many connected components is your network composed of? What is the size of the largest connected component? Run the ``find_components`` function to determine your answer. 

In [ ]:
components_adjacency = find_components(adjacency)
print('Our network is composed of', len(components_adjacency), 'connected components.\n')
print('The size of the largest connected component is ', len(components_adjacency[0]))

In [ ]:
#TESTING
test_node = np.ndarray(shape=(1,3760), buffer = np.zeros(3760))
test_adjacency = np.concatenate((adjacency, test_node))
test_node = np.ndarray(shape=(3761,1), buffer = np.zeros(3761))
test_adjacency = np.concatenate((test_adjacency, test_node), axis = 1)

test_components_adjacency = find_components(test_adjacency)
print(len(test_components_adjacency[0]), len(test_components_adjacency[1]))

### Question 9

Write a function that takes as input the adjacency matrix and a node (`source`) and returns the length of the shortest path between that node and all nodes in the graph using Dijkstra's algorithm. **For the purposes of this assignment we are interested in the hop distance between nodes, not in the sum of weights. **

Hint: You might want to mask the adjacency matrix in the function ``compute_shortest_path_lengths`` in order to make sure you obtain a binary adjacency matrix. 

In [ ]:
test_array = np.zeros(5, dtype=int)
test_array[3] = 1
test_array

In [ ]:
def compute_shortest_path_lengths(adjacency, source):
    
    shortest_path_lengths = np.zeros(n_nodes, dtype=int)
    visited, queue = set(), [source]
    hop_distance = 0
    queue2 = list()
    while queue:
        index = queue.pop(0)
        if index not in visited:
            visited.add(index)
            shortest_path_lengths[index] = hop_distance
            actor_connected_to = set(np.nonzero(adjacency[index])[0])
            queue2.extend(actor_connected_to - visited)
            if not queue:
                hop_distance = hop_distance + 1
                queue.extend(queue2)
                queue2 = list()    
    return shortest_path_lengths

### Question 10

The diameter of the graph is the length of the longest shortest path between any pair of nodes. Use the above developed function to compute the diameter of the graph (or the diameter of the largest connected component of the graph if the graph is not connected). If your graph (or largest connected component) is very large, computing the diameter will take very long. In that case downsample your graph so that it has 1.000 nodes. There are many ways to reduce the size of a graph. For the purposes of this milestone you can chose to randomly select 1.000 nodes. 

In [ ]:
# Your code here.
asdsa = compute_shortest_path_lengths(adjacency, 0)

In [ ]:
test_path_adjacency = np.ndarray((5,5),zeros)


### Question 11

Write a function that takes as input the adjacency matrix, a path length, and two nodes (`source` and `target`), and returns the number of paths of the given length between them.

In [ ]:
def compute_paths(adjacency, source, target, length):
    """Compute the number of paths of a given length between a source and target node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    target: int
        The target node. A number between 0 and n_nodes-1.
    length: int
        The path length to be considered.
    
    Returns
    -------
    int
        The number of paths.
    """
    
    # Your code here.
    
    return n_paths

Test your function on 5 pairs of nodes, with different lengths.

In [ ]:
print(compute_paths(adjacency, 0, 10, 1))
print(compute_paths(adjacency, 0, 10, 2))
print(compute_paths(adjacency, 0, 10, 3))
print(compute_paths(adjacency, 23, 67, 2))
print(compute_paths(adjacency, 15, 93, 4))

### Question 12

How many paths of length 3 are there in your graph? Hint: calling the `compute_paths` function on every pair of node is not an efficient way to do it.

In [ ]:
# Your code here.

### Question 13

Write a function that takes as input the adjacency matrix of your graph (or of the largest connected component of your graph) and a node and returns the clustering coefficient of that node. 

In [ ]:
def compute_clustering_coefficient(adjacency, node):
    """Compute the clustering coefficient of a node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    node: int
        The node whose clustering coefficient will be computed. A number between 0 and n_nodes-1.
    
    Returns
    -------
    float
        The clustering coefficient of the node. A number between 0 and 1.
    """
    
    # Your code here.
    
    return clustering_coefficient

### Question 14

What is the average clustering coefficient of your graph (or of the largest connected component of your graph if your graph is disconnected)? Use the function ``compute_clustering_coefficient`` to determine your answer.

In [ ]:
# Your code here.